<a href="https://colab.research.google.com/github/ObjectMatrix/google-colab-notebook/blob/main/Underwriting_Actuaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open('/content/drive/MyDrive/secrets.json', 'r') as f:
    secrets = json.load(f)

In [ ]:
!pip install youtube-transcript-api
!pip install pytube
!pip install langchain

In [ ]:
!pip install langchain!pip install langchain

In [ ]:
!pip install pypdf

In [ ]:
import json

with open('/content/drive/MyDrive/quoteSummaryMeta.json', 'r') as f:
    qsmContent = json.load(f)
print (qsmContent)    

In [ ]:
!cat /content/drive/MyDrive/quoteSummaryMeta.json

In [18]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import os
import json
import openai

from langchain.agents import (
    create_json_agent,
    AgentExecutor
)
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

with open('/content/drive/MyDrive/secrets.json', 'r') as f:
    secrets = json.load(f)

KEY = secrets['SECRET_KEY']
MODEL = secrets['MODEL']
openai.api_key = KEY
OPENAI_API_KEY=KEY
with open("/content/drive/MyDrive/quoteSummaryMeta.json", 'r') as f:
    data = json.load(f)
json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)
os.environ["OPENAI_API_KEY"] = KEY
json_agent_executor = create_json_agent(
    llm=OpenAI(temperature=0),
    toolkit=json_toolkit,
    verbose=True
)

**Train the Model**

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)


**Evaluate the Model**

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print("Mean Squared Error:", mse)
print("R-squared:", r2)


In [ ]:
# Define a function to get LLM recommendations for underwriting
def get_recommendation(prompt):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.5,
    )
    return response.choices[0].text.strip()


# Example: Get a recommendation for a specific customer profile
customer_profile = {"age": 45, "gender": "male", "annual_mileage": 12000}

recommendation = get_recommendation(f"Based on the customer profile {customer_profile}, should we offer them a lower, standard, or higher premium?")

print("Recommendation:", recommendation)


In [ ]:
prompt = """Assess the risk level of insuring a potential customer based on the following information:
- Age: 28
- Gender: Male
- Location: Los Angeles, California
- Driving history: One speeding ticket in the past three years, no accidents
- Vehicle: 2018 Honda Civic

Please provide a risk assessment and the factors contributing to this assessment."""


response = get_recommendation(prompt)
print(response)